In [24]:
import spacy
import pandas as pd
nlp = spacy.load("en_core_web_lg")


def preprocess(text):
    doc = nlp(text)
    filtered_tokens = list()
    for token in doc:
        if  token.is_stop or  token.is_punct:
            continue
        filtered_tokens.append(token.lemma_)
    return " ".join(filtered_tokens)

real = pd.read_csv("./data/fake_real_news/True.csv")
fake = pd.read_csv("./data/fake_real_news/Fake.csv")

In [25]:
real = real.drop(["date","subject","title"],axis=1)
fake = fake.drop(["date","subject","title"],axis=1)


In [26]:
fake

,text
0,Donald Trump just couldn t wish all Americans ...
1,House Intelligence Committee Chairman Devin Nu...
2,"On Friday, it was revealed that former Milwauk..."
3,"On Christmas day, Donald Trump announced that ..."
4,Pope Francis used his annual Christmas Day mes...
...,...
23476,21st Century Wire says As 21WIRE reported earl...
23477,21st Century Wire says It s a familiar theme. ...
23478,Patrick Henningsen 21st Century WireRemember ...
23479,21st Century Wire says Al Jazeera America will...


In [61]:
joined_df = pd.DataFrame()
joined_df = pd.concat([real, fake])

# Add 'label' column based on the source dataframe
joined_df['label'] = ['real'] * len(real) + ['fake'] * len(fake)

# Update 'label' column values based on dataframe source
joined_df.loc[joined_df.index.isin(real["text"]), 'label'] = 1
joined_df.loc[joined_df.index.isin(fake["text"]), 'label'] = 0

new_df = joined_df.sample(frac = 1)
new_df = new_df.iloc[:5000]

In [62]:
new_df["vector"] = new_df['text'].apply(lambda x:nlp(x).vector)

In [64]:
from sklearn.model_selection import train_test_split
# X_train, X_test, y_train
X_train, X_test, y_train, y_test = train_test_split(
    new_df.vector,
    new_df.label,
    test_size=0.2,
    random_state=2022
)

In [68]:
import numpy as np 
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import MinMaxScaler

X_train_2d = np.stack(X_train)
X_test_2d = np.stack(X_test)

scaler = MinMaxScaler()
scaled_X_train = scaler.fit_transform(X_train_2d)
scaled_X_test = scaler.fit_transform(X_test_2d)

clf = MultinomialNB()
clf.fit(scaled_X_train, y_train)

MultinomialNB()

In [69]:
from sklearn.metrics import classification_report
y_pred = clf.predict(scaled_X_test)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

        fake       0.90      0.79      0.84       531
        real       0.79      0.90      0.84       469

    accuracy                           0.84      1000
   macro avg       0.85      0.85      0.84      1000
weighted avg       0.85      0.84      0.84      1000



In [83]:
from sklearn.neighbors import KNeighborsClassifier

clf_knn = KNeighborsClassifier(n_neighbors=3,metric="euclidean")

clf_knn.fit(X_train_2d,y_train)

y_pred_knn = clf_knn.predict(scaled_X_test)

print(classification_report(y_test, y_pred_knn))

              precision    recall  f1-score   support

        fake       0.53      1.00      0.69       531
        real       0.00      0.00      0.00       469

    accuracy                           0.53      1000
   macro avg       0.27      0.50      0.35      1000
weighted avg       0.28      0.53      0.37      1000



/Users/aryagirigoudar/anaconda3/envs/AI/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/aryagirigoudar/anaconda3/envs/AI/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/aryagirigoudar/anaconda3/envs/AI/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, m

In [159]:
test ='''
I'm Killed someone Yesterday, due to my anger issues.
'''

a = preprocess(test)

a = nlp(a)
a = a.vector
a.reshape(-1, 1)

clf.predict([a])




array(['real'], dtype='<U4')

In [136]:
X_train_2d[0].shape

(300,)